In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
import torch
import torch
import random
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
import numpy as np
import math
import h5py
import os

In [ ]:
im = Image.open('../input/celeba-dataset/img_align_celeba/img_align_celeba/000001.jpg')
plt.imshow(im)

In [ ]:
from torchvision.transforms import ToTensor
t = ToTensor()
imt = t(im)
imt.shape

In [ ]:
df = pd.read_csv('../input/celeba-dataset/list_attr_celeba.csv')
len(df)

In [ ]:
def load_img(filepath):
    img = Image.open(filepath)
    return img

In [ ]:
def is_image_file(filename):
    return any(filename.endswith(extension) for extension in ['.png', '.jpg', '.jpeg'])

In [ ]:
import torch.utils.data as data
import torchvision
from os import listdir
class DatasetFromFolder(data.Dataset):
    def __init__(self, image_dir, LR_transform=None, HR_2_transform=None, 
                                 HR_4_transform=None):
        super(DatasetFromFolder, self).__init__()
        self.image_filenames = [join(image_dir, x) for x in listdir(image_dir) if is_image_file(x)]

        self.LR_transform = LR_transform
        self.HR_2_transform = HR_2_transform
        self.HR_4_transform = HR_4_transform
#         self.HR_8_transform = HR_8_transform


    def __getitem__(self, index):
        input = load_img(self.image_filenames[index])
        #print type(input)
#         HR_8 = self.HR_8_transform(input)
        #print type(HR_8)
        HR_4 = self.HR_4_transform(input)
        HR_2 = self.HR_2_transform(HR_4)
        LR = self.LR_transform(HR_4)
        to_tensor = torchvision.transforms.ToTensor()
        HR_4 = to_tensor(HR_4)
        return LR, HR_2, HR_4

    def __len__(self):
        return len(self.image_filenames)

In [ ]:
def get_training_set():
    root_dir = '../input/celeba-dataset'
    train_dir = join(root_dir, "img_align_celeba/img_align_celeba")

    return DatasetFromFolder(train_dir,
                             LR_transform=LR_transform(crop_size),
                             HR_2_transform=HR_2_transform(crop_size),
                             HR_4_transform=HR_4_transform(crop_size))



In [ ]:
def LR_transform(crop_size):
    return Compose([
        Scale(crop_size//4),
        ToTensor(),
    ])

def HR_2_transform(crop_size):
    return Compose([
        Scale(crop_size//2),
        ToTensor(),
    ])

def HR_4_transform(crop_size):
    return Compose([
        RandomCrop((crop_size, crop_size)),
        RandomHorizontalFlip(),
    ])

In [ ]:
from os.path import exists, join, basename
from os import makedirs, remove
from six.moves import urllib
import tarfile
from torchvision.transforms import Compose, CenterCrop, ToTensor, Scale, RandomCrop, RandomHorizontalFlip
crop_size =128

In [ ]:
train_set = get_training_set()

In [ ]:
len(train_set)

In [ ]:
from torch.utils.data.dataset import random_split
train_len = 200000
valid_len = 2599
TrainData1, ValidationData1 = random_split(train_set,[train_len, valid_len])

In [ ]:
from torch.utils.data import DataLoader
training_data_loader = DataLoader(dataset = TrainData1, num_workers = 1, batch_size = 64, shuffle = True)
testing_data_loader = DataLoader(dataset=ValidationData1, num_workers=1, batch_size=64, shuffle=False)

In [ ]:
from PIL import Image
import numpy as np
import random
import matplotlib.pyplot as plt
a = training_data_loader.dataset[0]

In [ ]:
import torchvision.transforms as transforms
plt.imshow(transforms.ToPILImage()(a[0]), interpolation="bicubic")

In [ ]:
class L1_Charbonnier_loss(nn.Module):
    """L1 Charbonnierloss."""
    def __init__(self):
        super(L1_Charbonnier_loss, self).__init__()
        self.eps = 1e-3

    def forward(self, X, Y):
        diff = torch.add(X, -Y)
        error = torch.sqrt( diff * diff + self.eps * self.eps )
        loss = torch.sum(error) 
        return loss

In [ ]:
def get_upsample_filter(size):
    """Make a 2D bilinear kernel suitable for upsampling"""
    factor = (size + 1) // 2
    if size % 2 == 1:
        center = factor - 1
    else:
        center = factor - 0.5
    og = np.ogrid[:size, :size]
    filter = (1 - abs(og[0] - center) / factor) * \
             (1 - abs(og[1] - center) / factor)
    return torch.from_numpy(filter).float()

In [ ]:
class _Conv_Block(nn.Module):    
    def __init__(self):
        super(_Conv_Block, self).__init__()
        
        self.cov_block = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose2d(in_channels=64, out_channels=64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
        )
        
    def forward(self, x):  
        output = self.cov_block(x)
        return output

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv_input = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False)
        self.relu = nn.LeakyReLU(0.2, inplace=True)
        
        self.convt_I1 = nn.ConvTranspose2d(in_channels=3, out_channels=3, kernel_size=4, stride=2, padding=1, bias=False)
        self.convt_R1 = nn.Conv2d(in_channels=64, out_channels=3, kernel_size=3, stride=1, padding=1, bias=False)
        self.convt_F1 = self.make_layer(_Conv_Block)
  
        self.convt_I2 = nn.ConvTranspose2d(in_channels=3, out_channels=3, kernel_size=4, stride=2, padding=1, bias=False)
        self.convt_R2 = nn.Conv2d(in_channels=64, out_channels=3, kernel_size=3, stride=1, padding=1, bias=False)
        self.convt_F2 = self.make_layer(_Conv_Block)        
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            if isinstance(m, nn.ConvTranspose2d):
                c1, c2, h, w = m.weight.data.size()
                weight = get_upsample_filter(h)
                m.weight.data = weight.view(1, 1, h, w).repeat(c1, c2, 1, 1)
                if m.bias is not None:
                    m.bias.data.zero_()
                    
    def make_layer(self, block):
        layers = []
        layers.append(block())
        return nn.Sequential(*layers)

    def forward(self, x):    
#         x = x.cuda()
#         print(type(x))
        out = self.relu(self.conv_input(x))
        
        convt_F1 = self.convt_F1(out)
        convt_I1 = self.convt_I1(x)
        convt_R1 = self.convt_R1(convt_F1)
        HR_2x = convt_I1 + convt_R1
        HR_2x = HR_2x.cuda()
        
        convt_F2 = self.convt_F2(convt_F1)
        convt_I2 = self.convt_I2(HR_2x)
        convt_R2 = self.convt_R2(convt_F2)
        HR_4x = convt_I2 + convt_R2
        HR_4x = HR_4x.cuda()
        return HR_2x, HR_4x

In [ ]:
okay = iter(training_data_loader)
first = next(okay)
first[0].shape

In [ ]:
# plt.imshow(first[2][0][0].cpu(), cmap = 'gray')
plt.imshow(transforms.ToPILImage()(first[2][0]), interpolation="bicubic")

In [ ]:
class lapinit():
    def __init__(self, batchSize = 64, nEpochs = 100, lr = 1e-4, step = 100,
            cuda = {'action' : True}, resume = '', start_epoch = 1, threads = 1,
            momentum = 0.9, weight_decay = 1e-4, pretrained = ''):
        self.batchSize = batchSize
        self.nEpochs = nEpochs
        self.lr = lr
        self.step = step
        self.cuda = cuda
        self.resume = resume
        self.start_epoch = start_epoch
        self.threads = threads
        self.momentum = momentum
        self.weight_decay = weight_decay
        self.pretrained = pretrained

opt = lapinit()



In [ ]:
model = Net()
model = model.cuda()
criterion = L1_Charbonnier_loss()

In [ ]:
first[0] = first[0].cuda()
tmp1, tmp2 = model(first[0])
tmp1.shape, tmp2.shape

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=opt.lr)

In [ ]:
okay = iter(testing_data_loader)
test1 = next(okay)
test1[0].shape

In [ ]:
test1[0] = test1[0].cuda()

In [ ]:
img_list1 = []
img_list2 = []

In [ ]:
def train(training_data_loader, optimizer, model, criterion, epoch):

#     lr = adjust_learning_rate(optimizer, epoch-1)
#     lr = input()
    global lr
    lr = float(lr)

    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

    print("Epoch={}, lr={}".format(epoch, optimizer.param_groups[0]["lr"]))

    model.train()

    for iteration, batch in enumerate(training_data_loader, 1):

        input, label_x2, label_x4 = Variable(batch[0]), Variable(batch[1], requires_grad=False), Variable(batch[2], requires_grad=False)

        if opt.cuda['action']:
#             input = input.cuda()
#             label_x2 = label_x2.cuda()
#             label_x4 = label_x4.cuda()
            
            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
            input, label_x2, label_x4 = input.to(device), label_x2.to(device), label_x4.to(device)
#             print(type(input))

        HR_2x, HR_4x = model(input)
#         HR_2x = HR_2x.cuda()
#         HR_4x = HR_4x.cuda()
        # print(HR_4x.shape)
        # print(label_x4.shape)
        loss_x2 = criterion(HR_2x, label_x2)
        loss_x4 = criterion(HR_4x, label_x4)
        loss = loss_x2 + loss_x4

        optimizer.zero_grad()

        loss_x2.backward(retain_graph = True)

        loss_x4.backward()

        optimizer.step()

        if iteration%100 == 0:
            print("===> Epoch[{}]({}/{}): Loss: {:.10f}".format(epoch, iteration, len(training_data_loader), loss.data.item()))
            tmp1, tmp2 = model(test1[0])
#             tmp1 = tmp1.cpu()
#             tmp2 = tmp2.cpu()
#             plt.imshow(transforms.ToPILImage()(tmp1[0].cpu()), interpolation="bicubic")
#             plt.show()
#             plt.imshow(transforms.ToPILImage()(tmp2[0].cpu()), interpolation="bicubic")
#             plt.show()
            img_list1.append(tmp1[0].detach().cpu())
            img_list2.append(tmp2[0].detach().cpu())
            del tmp1
            del tmp2
            torch.cuda.empty_cache()
#             tmp1.detach()
#             tmp2.detach()

In [ ]:
def save_checkpoint(model, epoch):
    model_folder = "checkpoint/"
    model_out_path = model_folder + "lapsrn_model_epoch_{}.pth".format(epoch)
    state = {"epoch": epoch ,"model": model, 'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict()}
    if not os.path.exists(model_folder):
        os.makedirs(model_folder)

    torch.save(state, model_out_path)

    print("Checkpoint saved to {}".format(model_out_path))

In [ ]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 10 epochs"""
#     if epoch == 2:
#         lr = 1e-3
#         return lr
    lr = opt.lr * (0.1 ** (epoch // (opt.step/10)))
    return lr

real image - 

In [ ]:
plt.imshow(transforms.ToPILImage()(test1[2][0].cpu()), interpolation="bicubic")

LR image of this is

In [ ]:
plt.imshow(transforms.ToPILImage()(test1[0][0].cpu()), interpolation="bicubic")
plt.show()

In [ ]:
from datetime import datetime
start=datetime.now()
for epoch in range(opt.start_epoch, opt.nEpochs + 1):
    lr = input()
    train(training_data_loader, optimizer, model, criterion, epoch)
    # save_checkpoint(model, epoch)
    print('do you want to save the model before proceeding?[y/n]')
    ch = input()
    if ch=='y':
        save_checkpoint(model, epoch)
    
    print(datetime.now() - start)

In [ ]:
checkpoint = torch.load('../working/checkpoint/lapsrn_model_epoch_7.pth')
model.load_state_dict(checkpoint['state_dict'])
model.eval()

In [ ]:
test1[0][0].shape

In [ ]:
# plt.imshow(test1[0][0][0].cpu(), cmap = 'gray')
plt.imshow(transforms.ToPILImage()(test1[0][0].cpu()), interpolation="bicubic")

In [ ]:
tmp1, tmp2 = model(test1[0])
tmp1 = tmp1.cpu()
tmp2 = tmp2.cpu()
plt.imshow(transforms.ToPILImage()(tmp1[0]), interpolation="bicubic")


In [ ]:
plt.imshow(transforms.ToPILImage()(tmp2[0]), interpolation="bicubic")

In [ ]:
# plt.imshow(tmp1[0][0].detach().cpu(), cmap = 'gray')
plt.imshow(transforms.ToPILImage()(tmp1[0].cpu()), interpolation="bicubic")

In [ ]:
# plt.imshow(tmp2[0][0].detach().cpu(), cmap = 'gray')
plt.imshow(transforms.ToPILImage()(tmp2[0].cpu()), interpolation="bicubic")

In [ ]:
# plt.imshow(test1[2][0][0].cpu(), cmap = 'gray')
plt.imshow(transforms.ToPILImage()(test1[2][0].cpu()), interpolation="bicubic")

## Animation of model(input)

In [ ]:
import matplotlib.animation as animation
from IPython.display import HTML
import torchvision.utils as vutils
fig = plt.figure(figsize = (6,6))
plt.axis('off')
# ims2 = [plt.imshow(torchvision.transforms.ToPILImage()(i.detach()), animated = True) for i in img_list2]
ims2 = [[plt.imshow(np.transpose(i.detach(), (1,2,0)), animated=True)] for i in img_list2]
ani = animation.ArtistAnimation(fig, ims2, interval = 100, repeat_delay = 500, blit = True)
HTML(ani.to_jshtml())


In [ ]:
fig = plt.figure(figsize = (6,6))
plt.axis('off')
# ims2 = [plt.imshow(torchvision.transforms.ToPILImage()(i.detach()), animated = True) for i in img_list2]
ims2 = [[plt.imshow(np.transpose(i.detach(), (1,2,0)), animated=True)] for i in img_list1]
ani = animation.ArtistAnimation(fig, ims2, interval = 100, repeat_delay = 500, blit = True)
HTML(ani.to_jshtml())

In [ ]:
# save_checkpoint(model, 1)

In [ ]:
test1[0].shape

In [ ]:
plt.imshow(torchvision.transforms.ToPILImage()(test1[0][0].cpu()), interpolation='bicubic')

In [ ]:
pred1 = model.conv_input(test1[0])
plt.imshow(torchvision.transforms.ToPILImage()(pred1[0].cpu()), interpolation='bicubic')

In [ ]:
import torch.nn.functional as F

In [ ]:
pred2 = F.relu(pred1)
# type(pred2)
plt.imshow(torchvision.transforms.ToPILImage()(pred2[0].cpu()), interpolation='bicubic')

In [ ]:
pred3 = model.convt_F1(pred2)
plt.imshow(torchvision.transforms.ToPILImage()(pred3[0].cpu()), interpolation='bicubic')

In [ ]:
pred4 = model.convt_I1(test1[0])
plt.imshow(torchvision.transforms.ToPILImage()(pred4[0].cpu()), interpolation='bicubic')

In [ ]:
pred5 = model.convt_R1(pred3)
plt.imshow(torchvision.transforms.ToPILImage()(pred5[0].cpu()), interpolation='bicubic')

In [ ]:
pred6 = pred4 + pred5
plt.imshow(torchvision.transforms.ToPILImage()(pred6[0].cpu()), interpolation='bicubic')